In [1]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 12.2 MB/s eta 0:00:00


In [2]:
!pip list

Package                       Version
----------------------------- --------------------
absl-py                       1.4.0
alabaster                     0.7.13
albumentations                1.2.1
alembic                       1.11.1
altair                        4.2.2
anyio                         3.6.2
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
array-record                  0.2.0
arviz                         0.15.1
astropy                       5.2.2
astunparse                    1.6.3
attrs                         23.1.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.12.1
backcall                      0.2.0
beautifulsoup4                4.11.2
bleach                        6.0.0
blis                          0.7.9
blosc2                        2.0.0
bokeh                         2.4.3
branca                        0.6.0
build                         0.10.0
Cach

In [3]:
import optuna

In [6]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [8]:
df = pd.read_csv("/content/Admission_Prediction.csv")

In [9]:
df.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,485.000000,490.000000,485.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.558763,107.187755,3.121649,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.274704,6.112899,1.146160,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,308.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,317.000000,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,325.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


In [10]:
df.isnull().sum()

Serial No.            0
GRE Score            15
TOEFL Score          10
University Rating    15
SOP                   0
LOR                   0
CGPA                  0
Research              0
Chance of Admit       0
dtype: int64

In [12]:
df["GRE Score"]=df["GRE Score"].fillna(df["GRE Score"].median())
df["TOEFL Score"]=df["TOEFL Score"].fillna(df["TOEFL Score"].median())
df["University Rating"]=df["University Rating"].fillna(df["University Rating"].median())


In [13]:
df.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [14]:
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,317.0,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65


In [15]:
X = df.drop(["Serial No.","Chance of Admit"], axis =1 )

In [16]:
y = df["Chance of Admit"]

In [17]:
X.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337.0,118.0,4.0,4.5,4.5,9.65,1
1,324.0,107.0,4.0,4.0,4.5,8.87,1
2,317.0,104.0,3.0,3.0,3.5,8.00,1
3,322.0,110.0,3.0,3.5,2.5,8.67,1
4,314.0,103.0,2.0,2.0,3.0,8.21,0


In [18]:
y.head()

0    0.92
1    0.76
2    0.72
3    0.80
4    0.65
Name: Chance of Admit, dtype: float64

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=25)

In [21]:
X_train.shape

(375, 7)

In [22]:
X_test.shape

(125, 7)

In [19]:
std_sca = StandardScaler()

In [23]:
X_train = std_sca.fit_transform(X_train)

In [24]:
X_test = std_sca.transform(X_test)

In [25]:
X_test.shape

(125, 7)

In [27]:
!nvidia-smi

Sun May 21 11:22:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [51]:
def objective(trail, data = X, target = y):
  train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.25, random_state=30)
  param={
      "tree_method":"gpu_hist",
      'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
      'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),
      'colsample_bytree' :trail.suggest_categorical('colsample_bytree' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
      'subsample' :trail.suggest_categorical('subsample' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
      'learning_rate' : trail.suggest_categorical('learning_rate' , [.00001,.0003,.008,.02,.01,1,8]),
      'n_estimator' :3000,
      'max_depth' :trail.suggest_categorical('max_depth', [3,4,5,6,7,8,9,10,11,12]),
      'random_state' :trail.suggest_categorical('random_state' , [10,20,30 ,2000 , 3454,243123]),
      'min_child_weight' :trail.suggest_int('min_child_weight' ,1,200)
       
  }
  xgb_reg_model = xgb.XGBRegressor(**param)
  xgb_reg_model.fit(train_x, train_y, eval_set = [(test_x, test_y)], verbose=True)
  pred_xgb = xgb_reg_model.predict(test_x)
  mse = mean_squared_error(test_y,pred_xgb)
  return mse

In [52]:
find_params=optuna.create_study()
find_params.optimize(objective,n_trials=10)
find_params.best_trial.params

[I 2023-05-21 12:23:55,723] A new study created in memory with name: no-name-f01a8ffc-805c-49de-bd5e-c1b8c851aa4b


[12:23:55] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimator" } are not used.

[0]	validation_0-rmse:0.25609
[1]	validation_0-rmse:0.25461
[2]	validation_0-rmse:0.25309
[3]	validation_0-rmse:0.25166
[4]	validation_0-rmse:0.25026
[5]	validation_0-rmse:0.24879
[6]	validation_0-rmse:0.24738
[7]	validation_0-rmse:0.24593
[8]	validation_0-rmse:0.24466
[9]	validation_0-rmse:0.24338
[10]	validation_0-rmse:0.24199
[11]	validation_0-rmse:0.24077
[12]	validation_0-rmse:0.23942
[13]	validation_0-rmse:0.23820
[14]	validation_0-rmse:0.23700
[15]	validation_0-rmse:0.23579
[16]	validation_0-rmse:0.23443
[17]	validation_0-rmse:0.23315
[18]	validation_0-rmse:0.23201
[19]	validation_0-rmse:0.23078
[20]	validation_0-rmse:0.22960
[21]	validation_0-rmse:0.22842
[22]	validation_0-rmse:0.22725
[23]	validation_0-rmse:0.22613
[24]	validation_0-rmse:0.22492
[25]	validation_0-rmse:0.22388
[26]	validation_0-rmse:0.22271
[27]	validation_0-rmse:0.22161
[28]	validation_0-rmse:0.22049
[29]	validation_0-rmse

<ipython-input-51-9bdf1f84bb89>:5: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.

<ipython-input-51-9bdf1f84bb89>:6: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.



[51]	validation_0-rmse:0.19867
[52]	validation_0-rmse:0.19789
[53]	validation_0-rmse:0.19706
[54]	validation_0-rmse:0.19632
[55]	validation_0-rmse:0.19557
[56]	validation_0-rmse:0.19479
[57]	validation_0-rmse:0.19398
[58]	validation_0-rmse:0.19326
[59]	validation_0-rmse:0.19250
[60]	validation_0-rmse:0.19179
[61]	validation_0-rmse:0.19106
[62]	validation_0-rmse:0.19035
[63]	validation_0-rmse:0.18966
[64]	validation_0-rmse:0.18892
[65]	validation_0-rmse:0.18827
[66]	validation_0-rmse:0.18768
[67]	validation_0-rmse:0.18709
[68]	validation_0-rmse:0.18638
[69]	validation_0-rmse:0.18576
[70]	validation_0-rmse:0.18506
[71]	validation_0-rmse:0.18443
[72]	validation_0-rmse:0.18373
[73]	validation_0-rmse:0.18316
[74]	validation_0-rmse:0.18259
[75]	validation_0-rmse:0.18200
[76]	validation_0-rmse:0.18144
[77]	validation_0-rmse:0.18095
[78]	validation_0-rmse:0.18036
[79]	validation_0-rmse:0.17975
[80]	validation_0-rmse:0.17921
[81]	validation_0-rmse:0.17873
[82]	validation_0-rmse:0.17816
[83]	val

[I 2023-05-21 12:23:56,124] Trial 0 finished with value: 0.028802972545805296 and parameters: {'lambda': 0.0002751690580715945, 'alpha': 0.2218987515133722, 'colsample_bytree': 0.8, 'subsample': 0.4, 'learning_rate': 0.008, 'max_depth': 6, 'random_state': 10, 'min_child_weight': 98}. Best is trial 0 with value: 0.028802972545805296.


...

[12:24:00] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimator" } are not used.

[0]	validation_0-rmse:0.25754
[1]	validation_0-rmse:0.25754
[2]	validation_0-rmse:0.25754
[3]	validation_0-rmse:0.25754
[4]	validation_0-rmse:0.25754
[5]	validation_0-rmse:0.25754
[6]	validation_0-rmse:0.25754
[7]	validation_0-rmse:0.25754
[8]	validation_0-rmse:0.25754
[9]	validation_0-rmse:0.25754
[10]	validation_0-rmse:0.25754
[11]	validation_0-rmse:0.25754
[12]	validation_0-rmse:0.25754
[13]	validation_0-rmse:0.25754
[14]	validation_0-rmse:0.25754
[15]	validation_0-rmse:0.25754
[16]	validation_0-rmse:0.25754
[17]	validation_0-rmse:0.25754
[18]	validation_0-rmse:0.25754
[19]	validation_0-rmse:0.25754
[20]	validation_0-rmse:0.25754
[21]	validation_0-rmse:0.25754
[22]	validation_0-rmse:0.25754
[23]	validation_0-rmse:0.25754
[24]	validation_0-rmse:0.25754
[25]	validation_0-rmse:0.25754
[26]	validation_0-rmse:0.25754
[27]	validation_0-rmse:0.25754
[28]	validation_0-rmse:0.25754
[29]	validation_0-rmse

<ipython-input-51-9bdf1f84bb89>:5: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.

<ipython-input-51-9bdf1f84bb89>:6: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.



[37]	validation_0-rmse:0.25754
[38]	validation_0-rmse:0.25754
[39]	validation_0-rmse:0.25754
[40]	validation_0-rmse:0.25754
[41]	validation_0-rmse:0.25754
[42]	validation_0-rmse:0.25754
[43]	validation_0-rmse:0.25754
[44]	validation_0-rmse:0.25754
[45]	validation_0-rmse:0.25754
[46]	validation_0-rmse:0.25754
[47]	validation_0-rmse:0.25754
[48]	validation_0-rmse:0.25754
[49]	validation_0-rmse:0.25754
[50]	validation_0-rmse:0.25754
[51]	validation_0-rmse:0.25754
[52]	validation_0-rmse:0.25754
[53]	validation_0-rmse:0.25754
[54]	validation_0-rmse:0.25754
[55]	validation_0-rmse:0.25754
[56]	validation_0-rmse:0.25754
[57]	validation_0-rmse:0.25754
[58]	validation_0-rmse:0.25754
[59]	validation_0-rmse:0.25754
[60]	validation_0-rmse:0.25754
[61]	validation_0-rmse:0.25754
[62]	validation_0-rmse:0.25754
[63]	validation_0-rmse:0.25754
[64]	validation_0-rmse:0.25754
[65]	validation_0-rmse:0.25754
[66]	validation_0-rmse:0.25754
[67]	validation_0-rmse:0.25754
[68]	validation_0-rmse:0.25754
[69]	val

[I 2023-05-21 12:24:00,582] Trial 9 finished with value: 0.0663272 and parameters: {'lambda': 0.29045923063528717, 'alpha': 0.6899893749665936, 'colsample_bytree': 0.1, 'subsample': 0.2, 'learning_rate': 0.0003, 'max_depth': 4, 'random_state': 243123, 'min_child_weight': 194}. Best is trial 2 with value: 0.009828544406786079.


{'lambda': 0.00018186468700433537,
 'alpha': 0.00011651880890506122,
 'colsample_bytree': 0.1,
 'subsample': 0.5,
 'learning_rate': 0.02,
 'max_depth': 8,
 'random_state': 20,
 'min_child_weight': 60}

In [49]:
find_params.trails_dataframe()

In [53]:
optuna.visualization.plot_optimization_history(find_params)

In [54]:
optuna.visualization.plot_slice(find_params)

In [55]:
optuna.visualization.plot_contour(find_params, params=['alpha','lambda'])

In [56]:
best_params = {'lambda': 0.0006365885262794263,
 'alpha': 0.042973494683172464,
 'colsample_bytree': 0.7,
 'subsample': 0.6,
 'learning_rate': 0.01,
 'max_depth': 9,
 'random_state': 2000,
 'min_child_weight': 17}

In [57]:
model = xgb.XGBRegressor(**best_params)

In [58]:
model.fit(X_train,y_train)

XGBRegressor(alpha=0.042973494683172464, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.7, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=0.0006365885262794263,
             learning_rate=0.01, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=9,
             max_leaves=None, min_child_weight=17, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, ...)

In [59]:
y_pred = model.predict(X_test)

In [60]:
from sklearn.metrics import r2_score

In [61]:
r2_score(y_test, y_pred)

0.3765397500040235

In [50]:
from sklearn.ensemble import RandomForestRegressor
model2 = RandomForestRegressor()
model2.fit(X_train,y_train)
pred_y = model.predict(X_test)
r2_score(y_test,y_pred)

0.3765397500040235